In [ ]:
import os
import pandas as pd
import tweepy
import requests
import pygsheets
import json
from requests_oauthlib import OAuth1Session

In [ ]:
# get your Bearer toke from your environmental variable 
bearer_token = os.environ.get("BEARER_TOKEN")
access_token = os.environ.get("ACCESS_TOKEN")
access_token_secret = os.environ.get("ACCESS_TOKEN_SECRET")
consumer_secret = os.environ.get("CONSUMER_SECRET")
consumer_key = os.environ.get("CONSUMER_KEY")

#tokens
oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r

# Get request token
request_token_url = "https://api.twitter.com/oauth/request_token"
oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)

try:
    fetch_response = oauth.fetch_request_token(request_token_url)
except ValueError:
    print(
        "There may have been an issue with the consumer_key or consumer_secret you entered."
    )

resource_owner_key = fetch_response.get("oauth_token")
resource_owner_secret = fetch_response.get("oauth_token_secret")


# # Get authorization
base_authorization_url = "https://api.twitter.com/oauth/authorize"
authorization_url = oauth.authorization_url(base_authorization_url)
print("Please go here and authorize: %s" % authorization_url)
verifier = input("Paste the PIN here: ")

# Get the access token
access_token_url = "https://api.twitter.com/oauth/access_token"
oauth = OAuth1Session(
    consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=resource_owner_key,
    resource_owner_secret=resource_owner_secret,
    verifier=verifier,
)
oauth_tokens = oauth.fetch_access_token(access_token_url)

access_token = oauth_tokens["oauth_token"]
access_token_secret = oauth_tokens["oauth_token_secret"]


#set rule for the filtered stream endpoints
fields = "created_at,description,location,public_metrics"
query_params_user = {'user.fields':'id,created_at,description,location,public_metrics,username'}
username = "myekini"
user_lookup =  'https://api.twitter.com/2/users/by/username/%s' %username


#get rtweets response from my profile
def run_query(user_lookup, query_params_user):
    return requests.get(user_lookup, auth=bearer_oauth, params=query_params_user).json()['data']
    
user_response = run_query(user_lookup,query_params_user)

In [ ]:

def get_user_data():
    #Build User table
    final_dict = {
        'created_at' : [],
        'name' : [],
        'username' : [],
        'id': [],
        'description' : [],
        'followers_count' : [],
        'following_count' : [],
        'listed_count' : [],
        'tweet_count' : []
    }



    final_dict['created_at'].append(user_response['created_at'])
    final_dict['name'].append(user_response['name'])
    final_dict['username'].append(user_response['username'])
    final_dict['id'].append(user_response['id'])
    final_dict['description'].append(user_response['description'])
    final_dict['followers_count'].append(user_response['public_metrics']['followers_count'])
    final_dict['following_count'].append(user_response['public_metrics']['following_count'])
    final_dict['listed_count'].append(user_response['public_metrics']['listed_count'])
    final_dict['tweet_count'].append(user_response['public_metrics']['tweet_count'])
    return final_dict

get_user_data()



In [ ]:
def get_tweets():
    """ getting tweets relating to my account"""
    ids = user_response['id']
    tweet_lookup = 'https://api.twitter.com/2/users/%s/tweets'%ids

    query_params_tweets ={'tweet.fields':'author_id,created_at,id,lang,text'}
    return requests.get(tweet_lookup, auth=bearer_oauth, params=query_params_tweets).json()['data']
    

my_tweets = get_tweets()

def get_user_tweets():
    user_tweet = {
        'author_id': [],
        'created_at': [],
        'id': [],
        'lang': [],
        'tweets':[]
    }
    for tweets in my_tweets:
        user_tweet['author_id'].append(tweets['author_id'])
        user_tweet['created_at'].append(tweets['created_at'])
        user_tweet['id'].append(tweets['id'])
        user_tweet['lang'].append(tweets['lang'])
        user_tweet['tweets'].append(tweets['text'])

    return user_tweet

get_user_tweets()



In [60]:
import gspread
import pygsheets
import pandas as pd

sheet_name = 'twitter_users'
# Create empty dataframe
df = pd.DataFrame()

def write_in_sheet(sheet_name, data, append=True):
    #authorization
    gc = pygsheets.authorize(service_file='service_account.json')

    # Create dataframe from our data dict
    df = pd.DataFrame.from_dict(data)

    #open the google spreadsheet (where 'PY to Gsheet Test' is the name of my sheet)
    sh = gc.open(sheet_name)

    #select the first sheet 
    wks = sh[0]
    cells = wks.get_all_values(include_tailing_empty_rows=False, include_tailing_empty=False, returnas='matrix')
    last_row = len(cells)

    #For hanlde sheet size problem
    wks.rows = last_row + 1000

    #Append or update data
    if append == True:
      if last_row <= 0: 
          wks.set_dataframe(df,(1,1))
      else:
          wks.set_dataframe(df,(last_row + 1,1), copy_head=False)
    else:
      #update the sheet
      wks.set_dataframe(df,(1,1))

def get_twitter_data_source():
    user_data = get_user_data()
    tweets_data = get_user_tweets()
    
    write_in_sheet('twitter_users', user_data, append=False)
    write_in_sheet('twitter_tweets', tweets_data, append=False)
get_twitter_data_source()